In [15]:
!pip install openai-whisper ffmpeg torch

In [17]:
from google.colab import drive

def mount_google_drive():
    """Google Driveをマウントします"""
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")

# Google Driveのマウント
mount_google_drive()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully.


In [18]:
import os

def list_media_files(folder_path):
    """指定フォルダ内のMP4またはMP3ファイルをリストアップ"""
    return [
        os.path.join(folder_path, f)
        for f in os.listdir(folder_path)
        if f.endswith(('.mp4', '.mp3'))
    ]

# Google Drive内の対象フォルダを指定してください
target_folder = "/content/drive/My Drive/TranscriptionFolder"
media_files = list_media_files(target_folder)

if not media_files:
    print("対象のMP4またはMP3ファイルが見つかりませんでした。")
else:
    print(f"対象ファイル: {media_files}")


対象ファイル: ['/content/drive/My Drive/TranscriptionFolder/13期生_Lesson09_2024-11-17_12_05.mp4', '/content/drive/My Drive/TranscriptionFolder/13期生_Lesson09_2024-11-17_12_05.mp3']


In [19]:
import subprocess

def convert_mp4_to_mp3(mp4_file):
    """MP4ファイルをMP3ファイルに変換"""
    mp3_file = mp4_file.replace(".mp4", ".mp3")
    command = f'ffmpeg -i "{mp4_file}" -q:a 0 -map a "{mp3_file}" -y'
    subprocess.run(command, shell=True, check=True)
    print(f"Converted {mp4_file} to {mp3_file}")
    return mp3_file


In [20]:
import whisper

def transcribe_files(files, model):
    """ファイルごとにテキスト化を実行し、結果を保存"""
    for file_path in files:
        print(f"Processing: {file_path}")
        try:
            result = model.transcribe(file_path)
            output_file = file_path.replace('.mp3', '.txt')
            with open(output_file, "w") as f:
                f.write(result["text"])
            print(f"Saved transcription to: {output_file}")
        except Exception as e:
            print(f"Error processing {file_path}: {e}")


In [22]:
import whisper

# Whisperモデルをロード
# FP16を使用しない設定（CPU用）
model = whisper.load_model("small", device="cpu")  # 必要に応じてモデルサイズを変更

# モデルのすべてのパラメータをFP32（単精度浮動小数点）に変換
for param in model.parameters():
    param.data = param.data.to(dtype=torch.float32)
    if param.grad is not None:
        param.grad.data = param.grad.data.to(dtype=torch.float32)

# MP4ファイルを変換してMP3をリストアップ
converted_mp3_files = []
for media_file in media_files:
    if media_file.endswith(".mp4"):
        try:
            converted_mp3_files.append(convert_mp4_to_mp3(media_file))
        except subprocess.CalledProcessError as e:
            print(f"Error converting {media_file}: {e}")
    elif media_file.endswith(".mp3"):
        converted_mp3_files.append(media_file)

# テキスト化を実行
if converted_mp3_files:
    transcribe_files(converted_mp3_files, model)
else:
    print("テキスト化するファイルがありません。")


Converted /content/drive/My Drive/TranscriptionFolder/13期生_Lesson09_2024-11-17_12_05.mp4 to /content/drive/My Drive/TranscriptionFolder/13期生_Lesson09_2024-11-17_12_05.mp3
Processing: /content/drive/My Drive/TranscriptionFolder/13期生_Lesson09_2024-11-17_12_05.mp3


/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


KeyboardInterrupt: 